<a href="https://colab.research.google.com/github/raj-coding1/comment-analysis/blob/main/Exp_3_tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow dvc dagshub optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.8/467.8 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.8/438.8 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0

In [ ]:
import pandas as pd
# from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import mlflow
import optuna
import dvc
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/df.csv')
df.head()
df.dropna(axis = 0,inplace = True)
X = df['cleaned_comment']

y = df['category']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/raj-coding1/youtube-comment-analysis.mlflow')
dagshub.init(repo_owner='raj-coding1', repo_name='youtube-comment-analysis', mlflow=True)

mlflow.set_experiment('EXP-9: BOW-TF/IDF-W2VEC')


def objective(trial):
    try:
        max_feat = trial.suggest_int("max_features", 1000, 20000, step=2000)
        ngram_choice = trial.suggest_categorical("ngram_range", [(1,1), (1,2), (1,3)])

        vectorizer = TfidfVectorizer(
            max_features=max_feat,
            ngram_range=ngram_choice
        )

        X_train_vec = vectorizer.fit_transform(X_train)
        X_val_vec = vectorizer.transform(X_test)

        run_name = f"trial_feature{max_feat}_ngram{ngram_choice}"

        with mlflow.start_run(run_name=run_name):

            mlflow.set_tag("experiment", "CountVectorizer")
            mlflow.set_tag("model", "RandomForest")
            mlflow.log_param('Vectorizer', 'TfIDF')
            mlflow.log_param("max_features", max_feat)
            mlflow.log_param("ngram_range", str(ngram_choice))

            # Train model
            n_estimator = 100
            max_depth = 10
            model = RandomForestClassifier(n_estimators=n_estimator, max_depth=max_depth, random_state=42)
            model.fit(X_train_vec, y_train)

            y_pred = model.predict(X_val_vec)
            accuracy = accuracy_score(y_test, y_pred)

            # Correct metric logging
            mlflow.log_metric("accuracy", accuracy)

            # Correct classification report
            class_rep = classification_report(y_test, y_pred, output_dict=True)

            for label, info in class_rep.items():
                if isinstance(info, dict):
                    for metric, value in info.items():
                        mlflow.log_metric(f"{label}_{metric}", value)

            # Save confusion matrix
            conf_matrix = confusion_matrix(y_test, y_pred)
            sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
            plt.xlabel("Prediction")
            plt.ylabel("Actual")
            plt.title("Confusion Matrix")

            plt.savefig("ConfusionMatrix.png")
            mlflow.log_artifact("/content/ConfusionMatrix.png")
            plt.close()

            # Save dataset (if needed)
            df.to_csv("df1.csv", index=False)
            mlflow.log_artifact("/content/df1.csv")

            print(f"accuracy : {accuracy}")

            return accuracy

    except Exception as e:
        print("Trial failed!", e)
        return 0.0  # ensures Optuna never fails


      # print(f'accuracy : {accuracy}')

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best Params:", study.best_params)
print("Best Score:", study.best_value)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=b950a497-d784-407d-b05e-7c7c60b43a1f&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=d85989becff54da1d1ddc146d86c36a3a5c54ea19232e70833494076e3ff324a




Output()

Accessing as raj-coding1

Initialized MLflow to track repo "raj-coding1/youtube-comment-analysis"

...

accuracy : 0.6264830219555434
🏃 View run trial_feature7000_ngram(1, 3) at: https://dagshub.com/raj-coding1/youtube-comment-analysis.mlflow/#/experiments/8/runs/b2d1d5ebbb09429bbe06f22dae29d1f0
🧪 View experiment at: https://dagshub.com/raj-coding1/youtube-comment-analysis.mlflow/#/experiments/8


[I 2025-11-30 04:15:52,260] Trial 18 finished with value: 0.6264830219555434 and parameters: {'max_features': 7000, 'ngram_range': (1, 3)}. Best is trial 13 with value: 0.6372562389199509.
/usr/local/lib/python3.12/dist-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [1000, 20000] and step=2000, but the range is not divisible by `step`. It will be replaced with [1000, 19000].
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/optuna/distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (1, 1) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.12/dist-packages/optuna/distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (1, 2) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.12/dist-package

accuracy : 0.6372562389199509
🏃 View run trial_feature5000_ngram(1, 3) at: https://dagshub.com/raj-coding1/youtube-comment-analysis.mlflow/#/experiments/8/runs/b55b6127ecd140cea16eb277afd233a1
🧪 View experiment at: https://dagshub.com/raj-coding1/youtube-comment-analysis.mlflow/#/experiments/8


[I 2025-11-30 04:16:24,246] Trial 19 finished with value: 0.6372562389199509 and parameters: {'max_features': 5000, 'ngram_range': (1, 3)}. Best is trial 13 with value: 0.6372562389199509.


Best Params: {'max_features': 5000, 'ngram_range': (1, 3)}
Best Score: 0.6372562389199509
